## 기본설정 및 함수정의

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import unary_union
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

# 영등포역 위도, 경도
ydp = [37.51569886583137, 126.90768824079629]

# Point를만드는 함수
def make_point(x):
    try:
        return Point(x)
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])
    
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

c:\Users\HJK\anaconda3\envs\GIS\lib\site-packages\geopandas\_compat.py:124: UserWarning:

The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.

C:\Users\HJK\AppData\Local\Temp\ipykernel_20076\4035485422.py:23: DeprecationWarning:

Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using

#### 영등포구 행정경계 (출처 - 통계지리정보서비스 2023년 센서스용 행정구역경계(읍면동))

In [2]:
# shp to geodataframe convert
shapefile_path = "SBJ_2406_001/_census_data_2023_bnd_dong_bnd_dong_11190_2023_2023\\bnd_dong_11190_2023_2023_2Q.shp"
ydp_gdf = gpd.read_file(shapefile_path)
ydp_gdf = ydp_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환

#### 격자(매핑용)

In [3]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/19.영등포_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

#### 영등포구 상세 도로망

In [4]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_001/6.영등포_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : make_lin(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
roadsystem_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')

In [22]:
from shapely import wkt
# 화재취약지역 데이터프레임을 GeoPandas 데이터프레임으로 변환
firefighting_vulnerable_df = pd.read_csv('visualization/화재취약지역.csv')
# 문자열을 폴리곤 객체로 변환
firefighting_vulnerable_df['geometry'] = firefighting_vulnerable_df['geometry'].apply(wkt.loads)
firefighting_vulnerable_df = gpd.GeoDataFrame(firefighting_vulnerable_df, geometry='geometry')

# 소방차진입 난이도 높은지역 데이터프레임을 GeoPandas 데이터프레임으로 변환
firefighting_car_vulnerable_df = pd.read_csv('visualization/소방차진입 난이도 높은지역.csv')
# 문자열을 폴리곤 객체로 변환
firefighting_car_vulnerable_df['geometry'] = firefighting_car_vulnerable_df['geometry'].apply(wkt.loads)
firefighting_car_vulnerable_df = gpd.GeoDataFrame(firefighting_car_vulnerable_df, geometry='geometry')

# 공간 조인하여 겹치는 부분만 남김
intersection_gdf = gpd.overlay(firefighting_vulnerable_df, firefighting_car_vulnerable_df, how='intersection')

C:\Users\HJK\AppData\Local\Temp\ipykernel_20076\2243823324.py:15: UserWarning:

`keep_geom_type=True` in overlay resulted in 4898 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries



In [ ]:
# 맵 객체 생성 및 데이터 로드
install_map = KeplerGl(height=800)
install_map.add_data(data=intersection_gdf, name="소방시설물 설치위치(격자)")
install_map.add_data(data=ydp_gdf, name="영등포구 행정동 데이터")
install_map

In [29]:
intersection_gdf = intersection_gdf[['gid_1', 'geometry']]
intersection_gdf.rename(columns={'gid_1' : 'gid'}, inplace=True)
intersection_gdf.to_csv("소방시설물 설치위치(격자단위).csv", index=False)

C:\Users\HJK\AppData\Local\Temp\ipykernel_20076\1652623135.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

